In [447]:
#Aproximacao do match dos catalogos Wen e Redmapper

from __future__ import division
import numpy as np
try:
    from scipy.spatial import cKDTree as KDT
except ImportError:
    from scipy.spatial import KDTree as KDT
import numpy as np
import astropy
from astropy.io import ascii
from astropy.io import fits
from astropy import units as u

In [448]:
#Finds matches in one catalog to another.
#Parameters
#ra1 : array-like
    #Right Ascension in degrees of the first catalog
#dec1 : array-like
    #Declination in degrees of the first catalog (shape of array must match `ra1`)
#ra2 : array-like
    #Right Ascension in degrees of the second catalog
#dec2 : array-like
    #Declination in degrees of the second catalog (shape of array must match `ra2`)
#tol : float or None, optional
    #How close (in degrees) a match has to be to count as a match.  If None,
    #all nearest neighbors for the first catalog will be returned.
#nnearest : int, optional
    #The nth neighbor to find.  E.g., 1 for the nearest nearby, 2 for the
    #second nearest neighbor, etc.  Particularly useful if you want to get
    #the nearest *non-self* neighbor of a catalog.  To do this, use:
    #``spherematch(ra, dec, ra, dec, nnearest=2)``
    
#Returns
#idx1 : int array
    #Indecies into the first catalog of the matches. Will never be
    #larger than `ra1`/`dec1`.
#idx2 : int array
    #Indecies into the second catalog of the matches. Will never be
    #larger than `ra1`/`dec1`.
#ds : float array
    #Distance (in degrees) between the matches

In [449]:
print 'Reading files'
w = ascii.read('wen.csv') 
r = fits.open('redmapper.fits') 
r = r[1].data

Reading files


In [450]:
tol = 31
nnearest=1

In [451]:
ra1 = w['RAJ2000']*u.degree
dec1 = w['DEJ2000']*u.degree
ra2 = r['RA']*u.degree
dec2 = r['DEC']*u.degree

ra1 = ra1[0:26111]
dec1 = dec1[0:26111]

ra1 = np.array(ra1, copy=False)
dec1 = np.array(dec1, copy=False)
ra2 = np.array(ra2, copy=False)
dec2 = np.array(dec2, copy=False)

if ra1.shape != dec1.shape:
    raise ValueError('ra1 and dec1 do not match!')
if ra2.shape != dec2.shape:
    raise ValueError('ra2 and dec2 do not match!')

rar = np.radians(ra1)
decr = np.radians(dec1)

x1 = np.cos(rar) * np.cos(decr)
y1 = np.sin(rar) * np.cos(decr)
z1 = np.sin(decr)

print x1, y1, z1

coords1 = np.empty((x1.size, 3))
coords1[:, 0] = x1
coords1[:, 1] = y1
coords1[:, 2] = z1
    
rar = np.radians(ra2)
decr = np.radians(dec2)

x2 = np.cos(rar) * np.cos(decr)
y2 = np.sin(rar) * np.cos(decr)
z2 = np.sin(decr)

print x2, y2, z2

coords2 = np.empty((x2.size, 3))
coords2[:, 0] = x2
coords2[:, 1] = y2
coords2[:, 2] = z2

kdt = KDT(coords2)
if nnearest == 1:
    idxs2 = kdt.query(coords1)[1]
elif nnearest > 1:
    idxs2 = kdt.query(coords1, nnearest)[1][:, -1]
else:
    raise ValueError('invalid nnearest ' + str(nnearest))

from numpy import radians, degrees, sin, cos, arctan2, hypot

lambs = radians(ra1)
phis = radians(dec1)
lambf = radians(ra2)
phif = radians(dec2)

dlamb = lambf - lambs

numera = cos(phif) * sin(dlamb)
numerb = cos(phis) * sin(phif) - sin(phis) * cos(phif) * cos(dlamb)
numer = hypot(numera, numerb)
denom = sin(phis) * sin(phif) + cos(phis) * cos(phif) * cos(dlamb)
ds = degrees(arctan2(numer, denom))

idxs1 = np.arange(ra1.size)

if tol is not None:
    msk = ds < tol
    idxs1 = idxs1[msk]
    idxs2 = idxs2[msk]
    ds = ds[msk]

print idxs1, idxs2, ds

print len(idxs1), len(idxs2), len(ds)

a = len(idxs1)

[ 0.84610713  0.99574357  0.85507704 ..., -0.08016235 -0.08067019
 -0.02556993] [  3.48509582e-05   1.66317069e-04   2.05502241e-04 ...,   3.91056559e-01
   3.93252351e-01   1.24275234e-01] [ 0.53301288  0.09216673  0.51850092 ...,  0.916869    0.91588477
  0.99191827]
[-0.45016287 -0.23358505 -0.95147866 ..., -0.61048977 -0.64952234
 -0.69374038] [-0.76677279 -0.644658   -0.30682337 ..., -0.00885283  0.31926356
  0.67079492] [ 0.45761653  0.72791077 -0.02340476 ...,  0.79197466  0.6900663
  0.26221835]
[    5    11    31 ..., 26030 26076 26089] [23322  8922 20660 ..., 26030 12073 17006] [ 23.04968019  13.75426717  16.12542867 ...,  10.5404787    9.86033387
  19.88342611]
3443 3443 3443


In [452]:
ra1 = w['RAJ2000']*u.degree
dec1 = w['DEJ2000']*u.degree
ra2 = r['RA']*u.degree
dec2 = r['DEC']*u.degree

ra1 = ra1[26111:52222]
dec1 = dec1[26111:52222]

ra1 = np.array(ra1, copy=False)
dec1 = np.array(dec1, copy=False)
ra2 = np.array(ra2, copy=False)
dec2 = np.array(dec2, copy=False)

if ra1.shape != dec1.shape:
    raise ValueError('ra1 and dec1 do not match!')
if ra2.shape != dec2.shape:
    raise ValueError('ra2 and dec2 do not match!')

rar = np.radians(ra1)
decr = np.radians(dec1)

x1 = np.cos(rar) * np.cos(decr)
y1 = np.sin(rar) * np.cos(decr)
z1 = np.sin(decr)

print x1, y1, z1

coords1 = np.empty((x1.size, 3))
coords1[:, 0] = x1
coords1[:, 1] = y1
coords1[:, 2] = z1
    
rar = np.radians(ra2)
decr = np.radians(dec2)

x2 = np.cos(rar) * np.cos(decr)
y2 = np.sin(rar) * np.cos(decr)
z2 = np.sin(decr)

print x2, y2, z2

coords2 = np.empty((x2.size, 3))
coords2[:, 0] = x2
coords2[:, 1] = y2
coords2[:, 2] = z2

kdt = KDT(coords2)
if nnearest == 1:
    idxs2 = kdt.query(coords1)[1]
elif nnearest > 1:
    idxs2 = kdt.query(coords1, nnearest)[1][:, -1]
else:
    raise ValueError('invalid nnearest ' + str(nnearest))

from numpy import radians, degrees, sin, cos, arctan2, hypot

lambs = radians(ra1)
phis = radians(dec1)
lambf = radians(ra2)
phif = radians(dec2)

dlamb = lambf - lambs

numera = cos(phif) * sin(dlamb)
numerb = cos(phis) * sin(phif) - sin(phis) * cos(phif) * cos(dlamb)
numer = hypot(numera, numerb)
denom = sin(phis) * sin(phif) + cos(phis) * cos(phif) * cos(dlamb)
ds = degrees(arctan2(numer, denom))

idxs1 = np.arange(ra1.size)

if tol is not None:
    msk = ds < tol
    idxs1 = idxs1[msk]
    idxs2 = idxs2[msk]
    ds = ds[msk]

print idxs1, idxs2, ds

print len(idxs1), len(idxs2), len(ds)

b = len(idxs1)

[-0.1608141  -0.14445494 -0.02006435 ..., -0.74449335 -0.9187328  -0.6831541 ] [ 0.77878339  0.69926355  0.09703557 ...,  0.28060272  0.34624556
  0.25744529] [ 0.60632933  0.70011661  0.99507865 ...,  0.60579845  0.18985273
  0.68339037]
[-0.45016287 -0.23358505 -0.95147866 ..., -0.61048977 -0.64952234
 -0.69374038] [-0.76677279 -0.644658   -0.30682337 ..., -0.00885283  0.31926356
  0.67079492] [ 0.45761653  0.72791077 -0.02340476 ...,  0.79197466  0.6900663
  0.26221835]
[    8    17    24 ..., 26095 26096 26108] [26030 19862 19862 ...,  7762 11345 21632] [ 26.56356767  10.94981403  22.58028494 ...,  22.09033416   9.70648696
  21.2830776 ]
4634 4634 4634


In [453]:
ra1 = w['RAJ2000']*u.degree
dec1 = w['DEJ2000']*u.degree
ra2 = r['RA']*u.degree
dec2 = r['DEC']*u.degree

ra1 = ra1[52222:78333]
dec1 = dec1[52222:78333]

ra1 = np.array(ra1, copy=False)
dec1 = np.array(dec1, copy=False)
ra2 = np.array(ra2, copy=False)
dec2 = np.array(dec2, copy=False)

if ra1.shape != dec1.shape:
    raise ValueError('ra1 and dec1 do not match!')
if ra2.shape != dec2.shape:
    raise ValueError('ra2 and dec2 do not match!')

rar = np.radians(ra1)
decr = np.radians(dec1)

x1 = np.cos(rar) * np.cos(decr)
y1 = np.sin(rar) * np.cos(decr)
z1 = np.sin(decr)

print x1, y1, z1

coords1 = np.empty((x1.size, 3))
coords1[:, 0] = x1
coords1[:, 1] = y1
coords1[:, 2] = z1
    
rar = np.radians(ra2)
decr = np.radians(dec2)

x2 = np.cos(rar) * np.cos(decr)
y2 = np.sin(rar) * np.cos(decr)
z2 = np.sin(decr)

print x2, y2, z2

coords2 = np.empty((x2.size, 3))
coords2[:, 0] = x2
coords2[:, 1] = y2
coords2[:, 2] = z2

kdt = KDT(coords2)
if nnearest == 1:
    idxs2 = kdt.query(coords1)[1]
elif nnearest > 1:
    idxs2 = kdt.query(coords1, nnearest)[1][:, -1]
else:
    raise ValueError('invalid nnearest ' + str(nnearest))

from numpy import radians, degrees, sin, cos, arctan2, hypot

lambs = radians(ra1)
phis = radians(dec1)
lambf = radians(ra2)
phif = radians(dec2)

dlamb = lambf - lambs

numera = cos(phif) * sin(dlamb)
numerb = cos(phis) * sin(phif) - sin(phis) * cos(phif) * cos(dlamb)
numer = hypot(numera, numerb)
denom = sin(phis) * sin(phif) + cos(phis) * cos(phif) * cos(dlamb)
ds = degrees(arctan2(numer, denom))

idxs1 = np.arange(ra1.size)

if tol is not None:
    msk = ds < tol
    idxs1 = idxs1[msk]
    idxs2 = idxs2[msk]
    ds = ds[msk]

print idxs1, idxs2, ds

print len(idxs1), len(idxs2), len(ds)

c = len(idxs1)

[-0.65204872 -0.87115817 -0.76759005 ..., -0.75454197 -0.6705428
 -0.93422231] [ 0.24568794  0.32815935  0.28913478 ..., -0.27899319 -0.24795816
 -0.34558674] [ 0.71726557  0.36523263  0.57201976 ...,  0.59399428  0.69920605
  0.08830896]
[-0.45016287 -0.23358505 -0.95147866 ..., -0.61048977 -0.64952234
 -0.69374038] [-0.76677279 -0.644658   -0.30682337 ..., -0.00885283  0.31926356
  0.67079492] [ 0.45761653  0.72791077 -0.02340476 ...,  0.79197466  0.6900663
  0.26221835]
[    7     9    16 ..., 26088 26090 26108] [ 4828 11345 20275 ..., 24310 11741 11741] [ 11.68552375  17.96223445  28.51701524 ...,  25.56952371  18.29162119
  21.00670873]
5976 5976 5976


In [454]:
ra1 = w['RAJ2000']*u.degree
dec1 = w['DEJ2000']*u.degree
ra2 = r['RA']*u.degree
dec2 = r['DEC']*u.degree

ra1 = ra1[78333:104444]
dec1 = dec1[78333:104444]

ra1 = np.array(ra1, copy=False)
dec1 = np.array(dec1, copy=False)
ra2 = np.array(ra2, copy=False)
dec2 = np.array(dec2, copy=False)

if ra1.shape != dec1.shape:
    raise ValueError('ra1 and dec1 do not match!')
if ra2.shape != dec2.shape:
    raise ValueError('ra2 and dec2 do not match!')

rar = np.radians(ra1)
decr = np.radians(dec1)

x1 = np.cos(rar) * np.cos(decr)
y1 = np.sin(rar) * np.cos(decr)
z1 = np.sin(decr)

print x1, y1, z1

coords1 = np.empty((x1.size, 3))
coords1[:, 0] = x1
coords1[:, 1] = y1
coords1[:, 2] = z1
    
rar = np.radians(ra2)
decr = np.radians(dec2)

x2 = np.cos(rar) * np.cos(decr)
y2 = np.sin(rar) * np.cos(decr)
z2 = np.sin(decr)

print x2, y2, z2

coords2 = np.empty((x2.size, 3))
coords2[:, 0] = x2
coords2[:, 1] = y2
coords2[:, 2] = z2

kdt = KDT(coords2)
if nnearest == 1:
    idxs2 = kdt.query(coords1)[1]
elif nnearest > 1:
    idxs2 = kdt.query(coords1, nnearest)[1][:, -1]
else:
    raise ValueError('invalid nnearest ' + str(nnearest))

from numpy import radians, degrees, sin, cos, arctan2, hypot

lambs = radians(ra1)
phis = radians(dec1)
lambf = radians(ra2)
phif = radians(dec2)

dlamb = lambf - lambs

numera = cos(phif) * sin(dlamb)
numerb = cos(phis) * sin(phif) - sin(phis) * cos(phif) * cos(dlamb)
numer = hypot(numera, numerb)
denom = sin(phis) * sin(phif) + cos(phis) * cos(phif) * cos(dlamb)
ds = degrees(arctan2(numer, denom))

idxs1 = np.arange(ra1.size)

if tol is not None:
    msk = ds < tol
    idxs1 = idxs1[msk]
    idxs2 = idxs2[msk]
    ds = ds[msk]

print idxs1, idxs2, ds

print len(idxs1), len(idxs2), len(ds)

d = len(idxs1)

[-0.87456486 -0.68021945 -0.9364123  ..., -0.4420539  -0.42510453
 -0.44168378] [-0.3236403  -0.2517211  -0.34652953 ..., -0.89692578 -0.86274782
 -0.89645178] [ 0.36110006  0.68843155  0.0552203  ...,  0.01060687  0.27377424
 -0.03591168]
[-0.45016287 -0.23358505 -0.95147866 ..., -0.61048977 -0.64952234
 -0.69374038] [-0.76677279 -0.644658   -0.30682337 ..., -0.00885283  0.31926356
  0.67079492] [ 0.45761653  0.72791077 -0.02340476 ...,  0.79197466  0.6900663
  0.26221835]
[    2    14    20 ..., 26091 26098 26102] [ 9203 11741 20042 ..., 14199  3233 24814] [  5.12174029  28.06768092  12.97388169 ...,  18.16262526  27.62281021
  11.78254374]
4992 4992 4992


In [455]:
ra1 = w['RAJ2000']*u.degree
dec1 = w['DEJ2000']*u.degree
ra2 = r['RA']*u.degree
dec2 = r['DEC']*u.degree

ra1 = ra1[104444:130555]
dec1 = dec1[104444:130555]

ra1 = np.array(ra1, copy=False)
dec1 = np.array(dec1, copy=False)
ra2 = np.array(ra2, copy=False)
dec2 = np.array(dec2, copy=False)

if ra1.shape != dec1.shape:
    raise ValueError('ra1 and dec1 do not match!')
if ra2.shape != dec2.shape:
    raise ValueError('ra2 and dec2 do not match!')

rar = np.radians(ra1)
decr = np.radians(dec1)

x1 = np.cos(rar) * np.cos(decr)
y1 = np.sin(rar) * np.cos(decr)
z1 = np.sin(decr)

print x1, y1, z1

coords1 = np.empty((x1.size, 3))
coords1[:, 0] = x1
coords1[:, 1] = y1
coords1[:, 2] = z1
    
rar = np.radians(ra2)
decr = np.radians(dec2)

x2 = np.cos(rar) * np.cos(decr)
y2 = np.sin(rar) * np.cos(decr)
z2 = np.sin(decr)

print x2, y2, z2

coords2 = np.empty((x2.size, 3))
coords2[:, 0] = x2
coords2[:, 1] = y2
coords2[:, 2] = z2

kdt = KDT(coords2)
if nnearest == 1:
    idxs2 = kdt.query(coords1)[1]
elif nnearest > 1:
    idxs2 = kdt.query(coords1, nnearest)[1][:, -1]
else:
    raise ValueError('invalid nnearest ' + str(nnearest))

from numpy import radians, degrees, sin, cos, arctan2, hypot

lambs = radians(ra1)
phis = radians(dec1)
lambf = radians(ra2)
phif = radians(dec2)

dlamb = lambf - lambs

numera = cos(phif) * sin(dlamb)
numerb = cos(phis) * sin(phif) - sin(phis) * cos(phif) * cos(dlamb)
numer = hypot(numera, numerb)
denom = sin(phis) * sin(phif) + cos(phis) * cos(phif) * cos(dlamb)
ds = degrees(arctan2(numer, denom))

idxs1 = np.arange(ra1.size)

if tol is not None:
    msk = ds < tol
    idxs1 = idxs1[msk]
    idxs2 = idxs2[msk]
    ds = ds[msk]

print idxs1, idxs2, ds

print len(idxs1), len(idxs2), len(ds)

e = len(idxs1)

[-0.44194514 -0.43806962 -0.20188205 ...,  0.96707909  0.9956553
  0.82822339] [-0.89703279 -0.88920487 -0.40983386 ..., -0.07333548 -0.07547415
 -0.06276754] [ 0.00408249  0.13194585  0.88953912 ...,  0.24368001  0.05453594
  0.55687185]
[-0.45016287 -0.23358505 -0.95147866 ..., -0.61048977 -0.64952234
 -0.69374038] [-0.76677279 -0.644658   -0.30682337 ..., -0.00885283  0.31926356
  0.67079492] [ 0.45761653  0.72791077 -0.02340476 ...,  0.79197466  0.6900663
  0.26221835]
[    0    10    13 ..., 26105 26106 26107] [14199  8968 18243 ...,  1561  1942 18256] [ 27.29767562  22.85456955  30.99805049 ...,  30.67527108  30.270391
  23.51786368]
3446 3446 3446


In [456]:
clusters = a + b + c + d + e 
print clusters

22491
